# Model Training

In [ ]:
import sys
import warnings
import pandas as pd
import numpy as np
import joblib
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import clone

sys.path.append("../../")

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

## Load Data

In [ ]:
df = pd.read_csv("../../data/insurance.csv")
df.head()

## Create Training and Test Sets

In [ ]:
mask = np.random.rand(len(df)) < 0.8

training_set = df[mask]

testing_set = df[~mask]

print(training_set.shape)
print(testing_set.shape)

In [ ]:
# save training and test sets to be used later
training_set.to_csv("../../data/training_set.csv")
testing_set.to_csv("../../data/testing_set.csv")

In [ ]:
# separating the feature columns from the target column
feature_columns = ["age", "sex", "bmi", "children", "smoker", "region"]
target_column = "charges"

X_train = training_set[feature_columns]
y_train = training_set[target_column]

X_test = testing_set[feature_columns]
y_test = testing_set[target_column]

## Apply the Preprocessing


In [ ]:
# loading the preprocessing pipeline we built in the previous notebook
transformer = joblib.load("transformer.joblib")

In [ ]:
# applying the column transformer
features = transformer.fit_transform(X_train)

features

## Find an Optimal Pipeline

In [ ]:
tpot_regressor = TPOTRegressor(generations=50,
                               population_size=50,
                               random_state=42,
                               cv=5,
                               n_jobs=8,
                               verbosity=2,
                               early_stop=10)

In [ ]:
tpot_regressor = tpot_regressor.fit(features, y_train)

## Create Pipeline

Now that we have an optimal pipeline created by TPOT we will be adding our own preprocessors to it. To do this we'll need to have an unfitted pipeline object.

To create an unfitted pipeline from the fitted pipeline that we already have, we'll clone the pipeline object:

In [ ]:
unfitted_tpot_regressor = clone(tpot_regressor.fitted_pipeline_)

unfitted_tpot_regressor

Now that we can build the same pipeline that was found by the TPOT package, we'll add our own preprocessors to the pipeline. This will ensure that the final pipeline will accept the features in the original dataset and will process the features correctly.

We'll compose the preprocessing pipeline and the tpot pipeline into one pipeline:

In [ ]:
model = Pipeline([
    ("transformer", transformer),
    ("tpot_pipeline", unfitted_tpot_regressor)
])

## Train Model

In [ ]:
model.fit(X_train, y_train)

## Test Model With Single Sample

In [ ]:
# testing the ColumnTransformer
test_df = pd.DataFrame([[65, "male", 12.5, 0, "yes", "southwest"]],
                       columns=["age", "sex", "bmi", "children", "smoker", "region"])


result = model.predict(test_df)

result

## Save Model

In [ ]:
joblib.dump(model, "model.joblib")